<a href="https://colab.research.google.com/github/Poonge1598/Unbiased_Toxicity_detection/blob/main/Toxicity_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'data-bal:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4285710%2F7569041%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240303%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240303T153632Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0654771a3c88bf3815ca7226b50d7a7ecacb4a5ec45149f6a6e9e36b46402122504d818ea7f33aa55f89705365399ba9f5b348d4336273ad518b640e54a589d386dc5215163fe04eabf8affa4404d01e24de6ae5ec13ca1c768e2bf71045c8a15eb0dba64389cce86d9fce8211755dcb8354d7c9427ea2c056874da3b688ef54ca2a08d444c420c4f01848907d40401cee687ff8a2b248d0014c0fb8de228fc731cec2971fe5583c6513c19a4b13b4b77f9d8d87e73374fde92fa54112720a1bf2a6abe0ffb867b377f7135e60fdf0bece8aca4f488824eb2e458cdb3d2437d1180263572151cc9a9dae95f64aa30e01a3ff633cde240bfe1bb58e67acb5c2fa'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
!pip install torchmetrics --quiet

In [ ]:
#Importing necessary packages
import os
import csv
import torch
import pickle
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm.notebook import tqdm
#from google.colab import drive
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from torchmetrics import AUROC, F1Score
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:

data_dir = '/kaggle/input/data-bal'

In [ ]:
torch.manual_seed(0)

In [ ]:
from gensim.models import KeyedVectors
glove_model = KeyedVectors.load("/kaggle/input/data-bal/glove-twitter-200.model", mmap='r')


In [ ]:
glove_model

## Creating the vocab for embedding matrix

In [ ]:
from symspellpy import SymSpell, Verbosity

# Initialize the SymSpell spell checker
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)

# Load the dictionary
sym_spell.load_dictionary('en-80k.txt', term_index=0, count_index=1)


ModuleNotFoundError: No module named 'symspellpy'

In [ ]:
import re
import string

misspell_dict = {"aren't": "are not", "can't": "cannot", "couldn't": "could not",
                 "didn't": "did not", "doesn't": "does not", "don't": "do not",
                 "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                 "he'd": "he would", "he'll": "he will", "he's": "he is",
                 "i'd": "I had", "i'll": "I will", "i'm": "I am", "isn't": "is not",
                 "it's": "it is", "it'll": "it will", "i've": "I have", "let's": "let us",
                 "mightn't": "might not", "mustn't": "must not", "shan't": "shall not",
                 "she'd": "she would", "she'll": "she will", "she's": "she is",
                 "shouldn't": "should not", "that's": "that is", "there's": "there is",
                 "they'd": "they would", "they'll": "they will", "they're": "they are",
                 "they've": "they have", "we'd": "we would", "we're": "we are",
                 "weren't": "were not", "we've": "we have", "what'll": "what will",
                 "what're": "what are", "what's": "what is", "what've": "what have",
                 "where's": "where is", "who'd": "who would", "who'll": "who will",
                 "who're": "who are", "who's": "who is", "who've": "who have",
                 "won't": "will not", "wouldn't": "would not", "you'd": "you would",
                 "you'll": "you will", "you're": "you are", "you've": "you have",
                 "'re": " are", "wasn't": "was not", "we'll": " will", "tryin'": "trying"}

In [ ]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']',
          '>', '%', '=', '#', '*', '+', '\\', '•', '~', '@', '£', '·', '_', '{', '}', '©', '^',
          '®', '`', '<', '→', '°', '€', '™', '›', '♥', '←', '×', '§', '″', '′', 'Â', '█',
          '½', 'à', '…', '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶',
          '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '▒', '：', '¼',
          '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲',
          'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', '∙', '）', '↓', '、', '│', '（', '»', '，', '♪',
          '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√']

In [ ]:
def _get_misspell(misspell_dict):
    misspell_re = re.compile('(%s)' % '|'.join(misspell_dict.keys()))
    return misspell_dict, misspell_re


def replace_typical_misspell(text):
    misspellings, misspellings_re = _get_misspell(misspell_dict)

    def replace(match):
        return misspellings[match.group(0)]

    return misspellings_re.sub(replace, text)


In [ ]:
import re
#from nltk.stem import WordNetLemmatizer
#from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

def clean_text(text):
    # Convert text to string
    text =str(text)

    # Convert text to lowercase
    text = text.lower()

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    #Strip blank spaces
    text=text.strip()

    text=replace_typical_misspell(text)

    # Initialize lemmatizer
    #lemmatizer = WordNetLemmatizer()
    #stop_words = set(stopwords.words("english"))

    # Lemmatize and remove stopwords
    #words = [lemmatizer.lemmatize(word) for word in text.split()]

    for punct in puncts + list(string.punctuation):

        if punct in text:
            text = text.replace(punct, f' {punct} ')


    #words=word_tokenize(text)



    # Spell checking and correction
    #corrected_words = []
    #for word in words:
    #    suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2, include_unknown=True)
    #    corrected_word = suggestions[0].term if suggestions else word
    #    corrected_words.append(corrected_word)

    # Remove non-alphanumeric characters and filter out single-character words
    #corrected_text = ' '.join(corrected_words)
    #corrected_text = re.sub(r'[^\w\s]', ' ', corrected_text)
    #final_words = [word for word in corrected_text.split() if len(word) > 1]

    # Rejoin corrected text
    #cleaned_text = ' '.join(final_words).strip()
    return text


In [ ]:
train_x = pd.read_csv(os.path.join(data_dir, 'train_x.csv'))
train_y = pd.read_csv(os.path.join(data_dir, 'train_y.csv'))
train_data = train_x.join(train_y)

val_x = pd.read_csv(os.path.join(data_dir, 'val_x.csv'))
val_y = pd.read_csv(os.path.join(data_dir, 'val_y.csv'))
val_data = val_x.join(val_y)

In [ ]:
ext_data = pd.read_csv(os.path.join(data_dir, 'ext_data.csv'))

In [ ]:
ext_data = ext_data.rename(columns = {

    'other_religion' : 'other_religions'
})

req_Cols = ['string', 'male', 'female', 'LGBTQ', 'christian', 'muslim',
       'other_religions', 'black', 'white', 'y']

train_data = train_data[req_Cols]
val_data = val_data[req_Cols]

train_data = pd.concat([train_data , ext_data], ignore_index = True)

In [ ]:
test_data = pd.read_csv(os.path.join(data_dir, 'test_x.csv'))

In [ ]:
train_data['string'] = train_data['string'].apply(clean_text)
val_data['string'] = val_data['string'].apply(clean_text)
test_data['string'] = test_data['string'].apply(clean_text)

In [ ]:
X_train = train_data['string'].fillna('_##_').values
X_test = test_data['string'].fillna('_##_').values
X_VAL = val_data['string'].fillna('_##_').values

In [ ]:
from itertools import chain
max_features = 100000
from collections import Counter


def build_vocab(texts, max_features):
    counter = Counter()
    for text in texts:
        counter.update(text.split())

    vocab = {
        'token2id': {'<PAD>': 0, '<UNK>': max_features + 1},
        'id2token': {}
    }
    vocab['token2id'].update(
        {token: _id + 1 for _id, (token, count) in
         enumerate(counter.most_common(max_features))})
    vocab['id2token'] = {v: k for k, v in vocab['token2id'].items()}
    return vocab

def tokenize(texts, vocab, max_len):
    pad_id = vocab['token2id']['<PAD>']  # ID for the <PAD> token
    unk_id = vocab['token2id']['<UNK>']  # ID for the <UNK> token

    def text2ids(text, token2id):
        # Convert text to IDs, using <UNK> ID if the token isn't found
        token_ids = [token2id.get(token, unk_id) for token in text.split()[:max_len]]

        # Pad the sequences to the maximum length
        token_ids += [pad_id] * (max_len - len(token_ids))
        return token_ids

    return [text2ids(text, vocab['token2id']) for text in texts]

In [ ]:
vocab = build_vocab(chain(X_train, X_test, X_VAL), max_features)

In [ ]:
X_train = np.array(tokenize(X_train, vocab, max_len=220))
X_test = np.array(tokenize(X_test, vocab, max_len=220))
X_val = np.array(tokenize(X_VAL, vocab, max_len=220))

In [ ]:
X_train

array([[101,  73, 126, ...,   0,   0,   0],
       [585, 129,   1, ...,   0,   0,   0],
       [ 15,  20,  51, ...,   0,   0,   0],
       ...,
       [ 14, 141,  29, ...,   0,   0,   0],
       [ 14,   8,   6, ...,   0,   0,   0],
       [ 72, 358, 475, ...,   0,   0,   0]])

In [ ]:
categories = ["male", "female", "LGBTQ", "christian", "muslim", "other_religions", "black", "white"]

In [ ]:
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.stem.lancaster import LancasterStemmer
import numpy as np

ps = PorterStemmer()
lc = LancasterStemmer()
sb = SnowballStemmer('english')


def load_embedding(embedding_path, word_index: dict, max_features: int, embed_size: int):

    def get_coefs(word, *arr):
        return word, np.asarray(arr, dtype='float32')

    embeddings_index = dict(get_coefs(*o.strip().split(' ')) for o in open(embedding_path))

    # word_index = tokenizer.word_index
    nb_words = min(max_features + 2, len(word_index))
    embedding_matrix = np.zeros((nb_words, embed_size))

    for key, i in word_index.items():
        if key in embeddings_index.keys():
            word = key
            embedding_vector = embeddings_index[word]
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
                continue
            word = key.lower()
            embedding_vector = embeddings_index[word]
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
                continue
            word = key.upper()
            embedding_vector = embeddings_index[word]
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
                continue
            word = key.capitalize()
            embedding_vector = embeddings_index[word]
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
                continue
            word = ps.stem(key)
            embedding_vector = embeddings_index[word]
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
                continue
            word = lc.stem(key)
            embedding_vector = embeddings_index[word]
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
                continue
            word = sb.stem(key)
            embedding_vector = embeddings_index[word]
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
                continue

    return embedding_matrix

In [ ]:
EMBEDDING_GLOVE='/kaggle/input/data-bal/glove.840B.300d.txt'

In [ ]:
glove_embedding_matrix = load_embedding(EMBEDDING_GLOVE, vocab['token2id'], max_features=max_features, embed_size=300)

In [ ]:
# Save the embedding matrix to a binary .npy file
np.save('embedding_matrix.npy', glove_embedding_matrix)

# To later load the embedding matrix:
# embedding_matrix = np.load('/mnt/data/embedding_matrix.npy')


In [ ]:
import numpy as np

# Assuming embedding_matrix is your numpy array containing the embeddings
num_zero_vectors = 0
for i, vector in enumerate(glove_embedding_matrix):
    if np.all(vector == 0):
        num_zero_vectors += 1

# Adjust for <PAD> and <UNK> if they are explicitly set to non-zero vectors
# If you've set <UNK> to a random or specific non-zero vector, adjust the count as follows:
# num_zero_vectors -= 1  # Adjust if <UNK> is not initialized to zero

print(f"Number of words not in GloVe model: {num_zero_vectors}")


Number of words not in GloVe model: 22295


In [ ]:
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(0)

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
device

device(type='cuda')

In [ ]:
train_data

,string,male,female,LGBTQ,christian,muslim,other_religions,black,white,y
0,even up here . . . . . . . ...,0,0,0,0,0,0,1,0,1
1,blame men . there is always an excuse to b...,1,1,0,0,0,0,0,0,1
2,you have no business making any comments on th...,0,0,0,0,0,0,0,0,1
3,""" let us get the black folks and the white ...",0,0,0,0,0,0,1,1,1
4,i guess the issue is people not willing to put...,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
329121,the conservative economics of expanded trade h...,0,0,0,0,0,0,0,1,0
329122,so when white supremacists erect confederate s...,0,0,0,0,0,0,0,1,0
329123,"it really was not about the niqab , was it ...",0,0,0,0,1,0,0,0,0
329124,it is of course normal and natural for eugene ...,0,0,0,0,0,0,0,1,0


In [ ]:
categories

['male',
 'female',
 'LGBTQ',
 'christian',
 'muslim',
 'other_religions',
 'black',
 'white']

In [ ]:
grouped_data=train_data[categories+['y']].groupby('y').sum()

In [ ]:
grouped_data

,male,female,LGBTQ,christian,muslim,other_religions,black,white
y,,,,,,,,
0,32133,36905,14383,29516,27521,5557,17575,31208
1,6053,6117,4270,3324,6099,1006,6190,9342


In [ ]:
grouped_data=grouped_data.transpose()

In [ ]:
grouped_data

y,0,1
male,32133,6053
female,36905,6117
LGBTQ,14383,4270
christian,29516,3324
muslim,27521,6099
other_religions,5557,1006
black,17575,6190
white,31208,9342


In [ ]:
grouped_data['weights']=(grouped_data[1]+grouped_data[0])/(grouped_data[0]-grouped_data[1])

In [ ]:
grouped_data

y,0,1,weights
male,32133,6053,1.464187
female,36905,6117,1.397363
LGBTQ,14383,4270,1.844458
christian,29516,3324,1.253818
muslim,27521,6099,1.569415
other_religions,5557,1006,1.442101
black,17575,6190,2.087396
white,31208,9342,1.854477


In [ ]:
weights={f'{i}_1':None for i in grouped_data.index}

In [ ]:
weights

{'male_1': None,
 'female_1': None,
 'LGBTQ_1': None,
 'christian_1': None,
 'muslim_1': None,
 'other_religions_1': None,
 'black_1': None,
 'white_1': None}

In [ ]:
main_category=['black','white']

In [ ]:
second_category=['LGBTQ','muslim']

In [ ]:
for i in grouped_data.index:
    if i in main_category:
        weights[f'{i}_1']=grouped_data.loc[i,'weights']*2
        weights[f'{i}_0']=1
    elif i in second_category:
        weights[f'{i}_1']=grouped_data.loc[i,'weights']
        weights[f'{i}_0']=1

    else:
        weights[f'{i}_1']=1
        weights[f'{i}_0']=1




In [ ]:
weights

{'male_1': 1,
 'female_1': 1,
 'LGBTQ_1': 1.8444576287946208,
 'christian_1': 1,
 'muslim_1': 1.569414620483615,
 'other_religions_1': 1,
 'black_1': 4.174791392182697,
 'white_1': 3.7089545412969906,
 'male_0': 1,
 'female_0': 1,
 'LGBTQ_0': 1,
 'christian_0': 1,
 'muslim_0': 1,
 'other_religions_0': 1,
 'black_0': 1,
 'white_0': 1}

In [ ]:
mask_male_toxic = (train_data['male'] > 0.5) & (train_data['y'] == 1)
mask_female_toxic = (train_data['female'] > 0.5) & (train_data['y'] == 1)
mask_LGBTQ_toxic = (train_data['LGBTQ'] > 0.5) & (train_data['y'] == 1)
mask_christian_toxic = (train_data['christian'] > 0.5) & (train_data['y'] == 1)
mask_muslim_toxic = (train_data['muslim'] > 0.5) & (train_data['y'] == 1)
mask_other_religions_toxic = (train_data['other_religions'] > 0.5) & (train_data['y'] == 1)
mask_black_toxic = (train_data['black'] > 0.5) & (train_data['y'] == 1)
mask_white_toxic = (train_data['white'] > 0.5) & (train_data['y'] == 1)

In [ ]:
train_data['weights'] = 1

In [ ]:
train_data['weights'] = 1
train_data.loc[mask_LGBTQ_toxic, 'weights'] += weights['LGBTQ_1']
train_data.loc[mask_muslim_toxic, 'weights'] += weights['muslim_1']
train_data.loc[mask_black_toxic, 'weights'] += weights['black_1']
train_data.loc[mask_white_toxic, 'weights'] += weights['white_1']

/tmp/ipykernel_35/991162300.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2.84445763 2.84445763 2.84445763 ... 2.84445763 2.84445763 2.84445763]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  train_data.loc[mask_LGBTQ_toxic, 'weights'] += weights['LGBTQ_1']


In [ ]:
train_data

,string,male,female,LGBTQ,christian,muslim,other_religions,black,white,y,weights
0,even up here . . . . . . . ...,0,0,0,0,0,0,1,0,1,5.174791
1,blame men . there is always an excuse to b...,1,1,0,0,0,0,0,0,1,1.000000
2,you have no business making any comments on th...,0,0,0,0,0,0,0,0,1,1.000000
3,""" let us get the black folks and the white ...",0,0,0,0,0,0,1,1,1,8.883746
4,i guess the issue is people not willing to put...,0,0,0,0,0,0,0,0,1,1.000000
...,...,...,...,...,...,...,...,...,...,...,...
329121,the conservative economics of expanded trade h...,0,0,0,0,0,0,0,1,0,1.000000
329122,so when white supremacists erect confederate s...,0,0,0,0,0,0,0,1,0,1.000000
329123,"it really was not about the niqab , was it ...",0,0,0,0,1,0,0,0,0,1.000000
329124,it is of course normal and natural for eugene ...,0,0,0,0,0,0,0,1,0,1.000000


In [ ]:
val_data['weights'] = 1

In [ ]:
class CommentDataset(Dataset):
    def __init__(self, comments, labels):
        self.comments = comments
        self.labels = labels

    def __len__(self):
        return len(self.comments)

    def __getitem__(self, idx):
        comment = torch.tensor(self.comments[idx], dtype=torch.long)
        label = torch.tensor(self.labels.loc[idx,'y'], dtype=torch.float)
        weights = torch.tensor(self.labels.loc[idx,'weights'], dtype = torch.float)


        return comment, label ,idx, weights

In [ ]:
# Create datasets
train_dataset = CommentDataset(X_train, train_data)  # Assuming train_labels is defined
val_dataset = CommentDataset(X_val, val_data)  # Assuming val_labels is defined

# Create data loaders
batch_size = 64  # Define your batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,pin_memory=True,num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size,pin_memory=True,num_workers=4)

In [ ]:
train_dataset[22]

(tensor([  14,  545,   68,  186,  275,    1,    5,   14,  536,  134,  156, 1432,
            3,   18,  104,    1,   95,   20,  222, 5426,   93,  433,  383,   13,
          181,  156,    5, 1985,   13,   70,  222,   46, 1742,   26, 5887,  176,
           30,  537,  225, 2657, 4879,   26, 1349,    3,   33,   70,  743,    4,
         1986,  932,  101,  384,   55,   17,  218,   59, 5680, 2378, 6051,   10,
         3143,   14,    1,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,  

In [ ]:
categories

['male',
 'female',
 'LGBTQ',
 'christian',
 'muslim',
 'other_religions',
 'black',
 'white']

In [ ]:
def initialize_weights(model):
    for m in model.modules():
        if isinstance(m, (nn.LSTM, nn.GRU)):
            # Initialize weights for LSTM/GRU
            for name, param in m.named_parameters():
                if 'weight_ih' in name:
                    torch.nn.init.xavier_uniform_(param.data)
                elif 'weight_hh' in name:
                    torch.nn.init.orthogonal_(param.data)
                elif 'bias' in name:
                    param.data.fill_(0)
        elif isinstance(m, nn.Linear):
            # Initialize weights for linear layers
            torch.nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0)

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, hidden_dim, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert hidden_dim % num_heads == 0, "hidden_dim must be divisible by num_heads"

        self.hidden_dim = hidden_dim
        self.num_heads = num_heads
        self.head_dim = hidden_dim // num_heads

        self.query = nn.Linear(hidden_dim, hidden_dim)
        self.key = nn.Linear(hidden_dim, hidden_dim)
        self.value = nn.Linear(hidden_dim, hidden_dim)

        self.fc_out = nn.Linear(hidden_dim, hidden_dim)

    def forward(self, x):
        batch_size = x.size(0)

        # Linear projections in batch from input to query, key, and value vectors
        query = self.query(x).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        key = self.key(x).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        value = self.value(x).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)

        # Scaled Dot-Product Attention
        energy = torch.matmul(query, key.transpose(-2, -1)) / (self.head_dim ** 0.5)
        attention = F.softmax(energy, dim=-1)  # Using softmax to compute attention weights
        out = torch.matmul(attention, value)

        # Concatenate heads
        out = out.transpose(1, 2).contiguous().view(batch_size, -1, self.hidden_dim)

        # Final linear layer
        out = self.fc_out(out)

        return out

In [ ]:
class GloVeClassifier_mul_att(nn.Module):
    # Initialize the classifier with various hyperparameters.
    def __init__(self, embedding_matrix: np.ndarray,hidden_dim=128, lstm_layers=2, gru_layers=1, dropout_rate_lstm=0, dropout_rate_gru=0, dropout_rate=0.1, embedding_dim=300, num_heads=8):

        super(GloVeClassifier_mul_att, self).__init__()
        self.embedding = nn.Embedding(*embedding_matrix.shape)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False

        self.embedding_dropout = nn.Dropout2d(0.1)
        # Bidirectional LSTM layer for capturing sequential information in both directions.
        self.bidirectional_lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=lstm_layers, bidirectional=True, dropout=dropout_rate_lstm, batch_first=True)
        # GRU layer to capture dependencies and handle vanishing gradient problem.
        self.gru = nn.GRU(hidden_dim * 2, hidden_dim // 2, num_layers=gru_layers, bidirectional=True, dropout=dropout_rate_gru, batch_first=True)
        # Multi-head attention mechanism for focusing on different parts of the sequence.
        self.multihead_attention = MultiHeadAttention(hidden_dim , num_heads)

        # Fully connected layers for classification.
        self.fc1 = nn.Linear(hidden_dim, hidden_dim //2)
        self.fc2 = nn.Linear(hidden_dim //2, 1)
        #self.fc3 = nn.Linear(hidden_dim, 1)
        # LeakyReLU activation function.
        self.leaky_relu = nn.LeakyReLU(negative_slope=0.01)

    # Define the forward pass of the network.
    def forward(self, x):
        h_embedding = self.embedding(x)
        h_embedding = torch.squeeze(torch.unsqueeze(h_embedding, 0))
        x, _ = self.bidirectional_lstm(h_embedding)  # LSTM processing
        x, _ = self.gru(x)                 # GRU processing
        x = self.multihead_attention(x)    # Apply multi-head attention
        #x = self.multihead_attention(x)
        x = x.mean(dim=1)

        x = self.leaky_relu(self.fc1(x))   # Fully connected layer with LeakyReLU
        #x = self.leaky_relu(self.fc2(x))   # Another fully connected layer
        x = self.fc2(x)
        #x = x.view(-1)  # Reshape output to match target shape [batch_size]

        return x


In [ ]:
embed_size = 300
maxlen=220



class Attention(nn.Module):
    def __init__(self, feature_dim, step_dim, bias=True, **kwargs):
        super(Attention, self).__init__(**kwargs)

        self.supports_masking = True

        self.bias = bias
        self.feature_dim = feature_dim
        self.step_dim = step_dim
        self.features_dim = 0

        weight = torch.zeros(feature_dim, 1)
        nn.init.kaiming_uniform_(weight)
        self.weight = nn.Parameter(weight)

        if bias:
            self.b = nn.Parameter(torch.zeros(step_dim))

    def forward(self, x, mask=None):
        feature_dim = self.feature_dim
        step_dim = self.step_dim
        eij = torch.mm(
            x.contiguous().view(-1, feature_dim),
            self.weight
        ).view(-1, step_dim)

        if self.bias:
            eij = eij + self.b

        eij = torch.tanh(eij)
        a = torch.exp(eij)

        if mask is not None:
            a = a * mask

        a = a / (torch.sum(a, 1, keepdim=True) + 1e-10)

        weighted_input = x * torch.unsqueeze(a, -1)
        return torch.sum(weighted_input, 1)

class Attention_Net(nn.Module):
    def __init__(self, embedding_matrix: np.ndarray):
        super(Attention_Net, self).__init__()
        drp = 0.1
        self.embedding = nn.Embedding(*embedding_matrix.shape)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False

        self.embedding_dropout = nn.Dropout2d(0.1)
        self.lstm = nn.LSTM(embed_size, 128, bidirectional=True, batch_first=True)
        self.lstm2 = nn.GRU(128*2, 64, bidirectional=True, batch_first=True)

        self.attention_layer = Attention(128, maxlen)

        self.linear = nn.Linear(64*2 , 64)
        self.relu = nn.ReLU()
        self.out = nn.Linear(64, 1)

    def forward(self, x):
        h_embedding = self.embedding(x)
        h_embedding = torch.squeeze(torch.unsqueeze(h_embedding, 0))
        h_lstm, _ = self.lstm(h_embedding)
        h_lstm, _ = self.lstm2(h_lstm)
        h_lstm_atten = self.attention_layer(h_lstm)
        conc = self.relu(self.linear(h_lstm_atten))
        out = self.out(conc)
        return out


In [ ]:
model = Attention_Net(glove_embedding_matrix)
initialize_weights(model)
model = nn.DataParallel(model).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0005)
#custom_loss=WeightedBCELoss(weights=weights,pos_weight=pos_weight).to(device)
#criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight).to(device)
#criterion=nn.BCELoss()
#metric = F1Score(task='binary').to(device)

NameError: name 'Attention_Net' is not defined

In [ ]:
model = GloVeClassifier_mul_att(glove_embedding_matrix)
initialize_weights(model)
model = nn.DataParallel(model).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0005)
#custom_loss=WeightedBCELoss(weights=weights,pos_weight=pos_weight).to(device)
#criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight).to(device)
#criterion=nn.BCELoss()
#metric = F1Score(task='binary').to(device)

In [ ]:
print(model)

DataParallel(
  (module): GloVeClassifier_mul_att(
    (embedding): Embedding(100002, 300)
    (embedding_dropout): Dropout2d(p=0.1, inplace=False)
    (bidirectional_lstm): LSTM(300, 128, num_layers=2, batch_first=True, bidirectional=True)
    (gru): GRU(256, 64, batch_first=True, bidirectional=True)
    (multihead_attention): MultiHeadAttention(
      (query): Linear(in_features=128, out_features=128, bias=True)
      (key): Linear(in_features=128, out_features=128, bias=True)
      (value): Linear(in_features=128, out_features=128, bias=True)
      (fc_out): Linear(in_features=128, out_features=128, bias=True)
    )
    (fc1): Linear(in_features=128, out_features=64, bias=True)
    (fc2): Linear(in_features=64, out_features=1, bias=True)
    (leaky_relu): LeakyReLU(negative_slope=0.01)
  )
)


In [ ]:
num_epochs=10

In [ ]:
import torchmetrics
metric =torchmetrics.Accuracy(task='binary').to(device)

In [ ]:
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=num_epochs, T_mult=1, eta_min=0.0001)


In [ ]:
def compute_group_accuracies(prediction, y):
    """
    Compute the accuracies for each group.
    arguments:
        prediction [pandas.DataFrame]: dataframe with 2 columns (index and pred)
        y [pandas.DataFrame]: dataframe containing the metadata
    returns:
        accuracies_dict [dict]: dictionary of accuracies for each group
        wga [float]: worst group accuracy
    """
    y.loc[prediction.index, 'pred'] = prediction.pred

    categories = ['male', 'female', 'LGBTQ', 'christian', 'muslim', 'other_religions', 'black', 'white']
    accuracies = []
    accuracies_dict = {}

    for category in categories:
        category_accuracies = []
        for label in [0, 1]:
            group = y.loc[y[category] == label]
            group_accuracy = (group['y'] == (group['pred'] > 0.5)).mean()
            category_accuracies.append(group_accuracy)
            accuracies_dict[f"{category}_{label}"] = group_accuracy
        accuracies.extend(category_accuracies)

    wga = np.min(accuracies)
    return accuracies_dict, wga


In [ ]:
# Training - model(Attention_Net)
for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0
    metric.reset()

    for x, y,idx,weights in tqdm(train_loader):
        x, y = x.to(device), y.to(device).squeeze()
        #print(weights)
        weights=weights.to(device)
        #print(x.shape,y.shape)
        #groups = [torch.tensor(group).to(device) for group in groups]
        optimizer.zero_grad()
        outputs = model(x).squeeze()
        #print(outputs.shape)
        #break
        #print(f"x shape: {x.shape}, y shape: {y.shape}, outputs shape: {outputs.shape}")
        #print(groups)
        #break
        #loss = custom_loss(outputs, y,groups)
        loss = (F.binary_cross_entropy_with_logits(outputs,y, reduction='none') * weights ).mean()
        loss.backward()
        # Gradient clipping
        #torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        #scheduler.step()

        total_train_loss += loss.item()
        #print(torch.sigmoid(outputs.squeeze()))
        #print(y.int())
        metric.update(torch.sigmoid(outputs), y.int())

    train_accuracy = metric.compute()
    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {total_train_loss/len(train_loader):.4f}, Training Accuracy: {train_accuracy:.4f}")

    # Validation
    model.eval()

    val_loss = 0.0
    metric.reset()
    val_predictions = []
    val_indices = []

    with torch.no_grad():
        for x, y,idx,weights in tqdm(val_loader):
            x, y = x.to(device), y.to(device).squeeze()
            #groups = [torch.tensor(group).to(device) for group in groups]
            outputs = model(x).squeeze()
            val_loss += (F.binary_cross_entropy_with_logits(outputs, y, reduction='none') ).mean().item()
            metric.update(torch.sigmoid(outputs), y.int())

            val_predictions.extend(torch.sigmoid(outputs).cpu().numpy())
            val_indices.extend(idx.numpy())

    val_accuracy = metric.compute()
    val_loss /= len(val_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")



    # Format predictions for WGA calculation
    val_pred_df = pd.DataFrame({'index': val_indices, 'pred': val_predictions}).set_index('index')

    # Compute WGA
    group_accuracies, wga = compute_group_accuracies(val_pred_df, val_loader.dataset.labels)  # Ensure val_metadata is defined and aligned
    print(f"Epoch {epoch+1}/{num_epochs}, Group Accuracies:")
    for group, accuracy in group_accuracies.items():
        print(f"  {group}: {accuracy:.4f}")
    print(f"Validation WGA: {wga:.4f}")

    accuracies_list = [accuracy for group, accuracy in group_accuracies.items()]
    median_accuracy = np.median(accuracies_list)
    mean_accuracy = np.mean(accuracies_list)
    print(f'median:{median_accuracy}')
    print(f'mean:{mean_accuracy}')

# Save the model if needed
# torch.save(model.state_dict(), 'model_path.pth')


  0%|          | 0/5143 [00:00<?, ?it/s]

Epoch 1/10, Training Loss: 0.3901, Training Accuracy: 0.8680


  0%|          | 0/706 [00:00<?, ?it/s]

Epoch 1/10, Validation Loss: 0.1998, Validation Accuracy: 0.9149
Epoch 1/10, Group Accuracies:
  male_0: 0.9181
  male_1: 0.8873
  female_0: 0.9181
  female_1: 0.8932
  LGBTQ_0: 0.9184
  LGBTQ_1: 0.8085
  christian_0: 0.9134
  christian_1: 0.9284
  muslim_0: 0.9197
  muslim_1: 0.8171
  other_religions_0: 0.9161
  other_religions_1: 0.8600
  black_0: 0.9237
  black_1: 0.6834
  white_0: 0.9279
  white_1: 0.7227
Validation WGA: 0.6834
median:0.9147544783053343
mean:0.8722592489675002


  0%|          | 0/5143 [00:00<?, ?it/s]

...

Epoch 9/10, Validation Loss: 0.2009, Validation Accuracy: 0.9308
Epoch 9/10, Group Accuracies:
  male_0: 0.9331
  male_1: 0.9108
  female_0: 0.9340
  female_1: 0.9092
  LGBTQ_0: 0.9333
  LGBTQ_1: 0.8552
  christian_0: 0.9297
  christian_1: 0.9407
  muslim_0: 0.9332
  muslim_1: 0.8815
  other_religions_0: 0.9317
  other_religions_1: 0.8915
  black_0: 0.9348
  black_1: 0.8257
  white_0: 0.9364
  white_1: 0.8483
Validation WGA: 0.8257
median:0.9306737162532579
mean:0.908059588483695


  0%|          | 0/5143 [00:00<?, ?it/s]

Epoch 10/10, Training Loss: 0.1446, Training Accuracy: 0.9529


  0%|          | 0/706 [00:00<?, ?it/s]

Epoch 10/10, Validation Loss: 0.2195, Validation Accuracy: 0.9337
Epoch 10/10, Group Accuracies:
  male_0: 0.9366
  male_1: 0.9093
  female_0: 0.9373
  female_1: 0.9094
  LGBTQ_0: 0.9360
  LGBTQ_1: 0.8655
  christian_0: 0.9329
  christian_1: 0.9407
  muslim_0: 0.9353
  muslim_1: 0.9009
  other_religions_0: 0.9345
  other_religions_1: 0.8986
  black_0: 0.9366
  black_1: 0.8565
  white_0: 0.9376
  white_1: 0.8758
Validation WGA: 0.8565
median:0.9336887309038858
mean:0.9152168415138764


In [ ]:
# Save the model's state, optimizer's state, and scheduler's state to a file.
torch.save({
    'epoch': epoch,  # Current epoch number
    'model_state_dict': model.module.state_dict(),  # State of the model
    'optimizer_state_dict': optimizer.state_dict(),  # State of the optimizer
    'loss': loss,  # Last recorded loss
    'scheduler_state_dict': scheduler.state_dict()  # State of the scheduler
}, 'glove_multi_head_attention_model_a.pth')


In [ ]:

for epoch in range(5):
    model.train()
    total_train_loss = 0
    metric.reset()

    for x, y,idx,weights in tqdm(train_loader):
        x, y = x.to(device), y.to(device).squeeze()
        #print(weights)
        weights=weights.to(device)
        #print(x.shape,y.shape)
        #groups = [torch.tensor(group).to(device) for group in groups]
        optimizer.zero_grad()
        outputs = model(x).squeeze()
        #print(outputs.shape)
        #break
        #print(f"x shape: {x.shape}, y shape: {y.shape}, outputs shape: {outputs.shape}")
        #print(groups)
        #break
        #loss = custom_loss(outputs, y,groups)
        loss = (F.binary_cross_entropy_with_logits(outputs,y, reduction='none') * weights ).mean()
        loss.backward()
        # Gradient clipping
        #torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        #scheduler.step()

        total_train_loss += loss.item()
        #print(torch.sigmoid(outputs.squeeze()))
        #print(y.int())
        metric.update(torch.sigmoid(outputs), y.int())

    train_accuracy = metric.compute()
    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {total_train_loss/len(train_loader):.4f}, Training Accuracy: {train_accuracy:.4f}")

    # Validation
    model.eval()

    val_loss = 0.0
    metric.reset()
    val_predictions = []
    val_indices = []

    with torch.no_grad():
        for x, y,idx,weights in tqdm(val_loader):
            x, y = x.to(device), y.to(device).squeeze()
            #groups = [torch.tensor(group).to(device) for group in groups]
            outputs = model(x).squeeze()
            val_loss += (F.binary_cross_entropy_with_logits(outputs, y, reduction='none') ).mean().item()
            metric.update(torch.sigmoid(outputs), y.int())

            val_predictions.extend(torch.sigmoid(outputs).cpu().numpy())
            val_indices.extend(idx.numpy())

    val_accuracy = metric.compute()
    val_loss /= len(val_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")



    # Format predictions for WGA calculation
    val_pred_df = pd.DataFrame({'index': val_indices, 'pred': val_predictions}).set_index('index')

    # Compute WGA
    group_accuracies, wga = compute_group_accuracies(val_pred_df, val_loader.dataset.labels)  # Ensure val_metadata is defined and aligned
    print(f"Epoch {epoch+1}/{num_epochs}, Group Accuracies:")
    for group, accuracy in group_accuracies.items():
        print(f"  {group}: {accuracy:.4f}")
    print(f"Validation WGA: {wga:.4f}")

    accuracies_list = [accuracy for group, accuracy in group_accuracies.items()]
    median_accuracy = np.median(accuracies_list)
    mean_accuracy = np.mean(accuracies_list)
    print(f'median:{median_accuracy}')
    print(f'mean:{mean_accuracy}')

# Save the model if needed
# torch.save(model.state_dict(), 'model_path.pth')


  0%|          | 0/5143 [00:00<?, ?it/s]

Epoch 1/10, Training Loss: 0.1313, Training Accuracy: 0.9583


  0%|          | 0/706 [00:00<?, ?it/s]

Epoch 1/10, Validation Loss: 0.2331, Validation Accuracy: 0.9254
Epoch 1/10, Group Accuracies:
  male_0: 0.9287
  male_1: 0.8976
  female_0: 0.9300
  female_1: 0.8948
  LGBTQ_0: 0.9273
  LGBTQ_1: 0.8682
  christian_0: 0.9243
  christian_1: 0.9352
  muslim_0: 0.9275
  muslim_1: 0.8834
  other_religions_0: 0.9262
  other_religions_1: 0.8915
  black_0: 0.9282
  black_1: 0.8529
  white_0: 0.9298
  white_1: 0.8601
Validation WGA: 0.8529
median:0.9252417275358459
mean:0.9066009053530012


  0%|          | 0/5143 [00:00<?, ?it/s]

...

Epoch 4/10, Validation Loss: 0.2744, Validation Accuracy: 0.9305
Epoch 4/10, Group Accuracies:
  male_0: 0.9332
  male_1: 0.9077
  female_0: 0.9346
  female_1: 0.9032
  LGBTQ_0: 0.9320
  LGBTQ_1: 0.8881
  christian_0: 0.9296
  christian_1: 0.9393
  muslim_0: 0.9317
  muslim_1: 0.9076
  other_religions_0: 0.9312
  other_religions_1: 0.9026
  black_0: 0.9326
  black_1: 0.8765
  white_0: 0.9336
  white_1: 0.8859
Validation WGA: 0.8765
median:0.9303632827987927
mean:0.9168401404455306


  0%|          | 0/5143 [00:00<?, ?it/s]

Epoch 5/10, Training Loss: 0.0923, Training Accuracy: 0.9724


  0%|          | 0/706 [00:00<?, ?it/s]

Epoch 5/10, Validation Loss: 0.2800, Validation Accuracy: 0.9289
Epoch 5/10, Group Accuracies:
  male_0: 0.9317
  male_1: 0.9047
  female_0: 0.9331
  female_1: 0.9009
  LGBTQ_0: 0.9304
  LGBTQ_1: 0.8840
  christian_0: 0.9278
  christian_1: 0.9385
  muslim_0: 0.9302
  muslim_1: 0.9019
  other_religions_0: 0.9298
  other_religions_1: 0.8864
  black_0: 0.9312
  black_1: 0.8674
  white_0: 0.9322
  white_1: 0.8800
Validation WGA: 0.8674
median:0.9288220327551165
mean:0.9131420123731123


In [ ]:
# Save the model's state, optimizer's state, and scheduler's state to a file.
torch.save({
    'epoch': epoch,  # Current epoch number
    'model_state_dict': model.module.state_dict(),  # State of the model
    'optimizer_state_dict': optimizer.state_dict(),  # State of the optimizer
    'loss': loss,  # Last recorded loss
    'scheduler_state_dict': scheduler.state_dict()  # State of the scheduler
}, 'glove_multi_head_attention_model_a_5.pth')


In [ ]:

for epoch in range(1):
    model.train()
    total_train_loss = 0
    metric.reset()

    for x, y,idx,weights in tqdm(train_loader):
        x, y = x.to(device), y.to(device).squeeze()
        #print(weights)
        weights=weights.to(device)
        #print(x.shape,y.shape)
        #groups = [torch.tensor(group).to(device) for group in groups]
        optimizer.zero_grad()
        outputs = model(x).squeeze()
        #print(outputs.shape)
        #break
        #print(f"x shape: {x.shape}, y shape: {y.shape}, outputs shape: {outputs.shape}")
        #print(groups)
        #break
        #loss = custom_loss(outputs, y,groups)
        loss = (F.binary_cross_entropy_with_logits(outputs,y, reduction='none') * weights ).mean()
        loss.backward()
        # Gradient clipping
        #torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        #scheduler.step()

        total_train_loss += loss.item()
        #print(torch.sigmoid(outputs.squeeze()))
        #print(y.int())
        metric.update(torch.sigmoid(outputs), y.int())

    train_accuracy = metric.compute()
    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {total_train_loss/len(train_loader):.4f}, Training Accuracy: {train_accuracy:.4f}")

    # Validation
    model.eval()

    val_loss = 0.0
    metric.reset()
    val_predictions = []
    val_indices = []

    with torch.no_grad():
        for x, y,idx,weights in tqdm(val_loader):
            x, y = x.to(device), y.to(device).squeeze()
            #groups = [torch.tensor(group).to(device) for group in groups]
            outputs = model(x).squeeze()
            val_loss += (F.binary_cross_entropy_with_logits(outputs, y, reduction='none') ).mean().item()
            metric.update(torch.sigmoid(outputs), y.int())

            val_predictions.extend(torch.sigmoid(outputs).cpu().numpy())
            val_indices.extend(idx.numpy())

    val_accuracy = metric.compute()
    val_loss /= len(val_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")



    # Format predictions for WGA calculation
    val_pred_df = pd.DataFrame({'index': val_indices, 'pred': val_predictions}).set_index('index')

    # Compute WGA
    group_accuracies, wga = compute_group_accuracies(val_pred_df, val_loader.dataset.labels)  # Ensure val_metadata is defined and aligned
    print(f"Epoch {epoch+1}/{num_epochs}, Group Accuracies:")
    for group, accuracy in group_accuracies.items():
        print(f"  {group}: {accuracy:.4f}")
    print(f"Validation WGA: {wga:.4f}")

    accuracies_list = [accuracy for group, accuracy in group_accuracies.items()]
    median_accuracy = np.median(accuracies_list)
    mean_accuracy = np.mean(accuracies_list)
    print(f'median:{median_accuracy}')
    print(f'mean:{mean_accuracy}')

# Save the model if needed
# torch.save(model.state_dict(), 'model_path.pth')


  0%|          | 0/5143 [00:00<?, ?it/s]

Epoch 1/10, Training Loss: 0.0866, Training Accuracy: 0.9744


  0%|          | 0/706 [00:00<?, ?it/s]

Epoch 1/10, Validation Loss: 0.2946, Validation Accuracy: 0.9309
Epoch 1/10, Group Accuracies:
  male_0: 0.9339
  male_1: 0.9060
  female_0: 0.9348
  female_1: 0.9054
  LGBTQ_0: 0.9325
  LGBTQ_1: 0.8854
  christian_0: 0.9299
  christian_1: 0.9402
  muslim_0: 0.9326
  muslim_1: 0.8981
  other_religions_0: 0.9318
  other_religions_1: 0.8915
  black_0: 0.9330
  black_1: 0.8759
  white_0: 0.9342
  white_1: 0.8835
Validation WGA: 0.8759
median:0.9308636681235558
mean:0.9155378894564121


In [ ]:
# Save the model's state, optimizer's state, and scheduler's state to a file.
torch.save({
    'epoch': epoch,  # Current epoch number
    'model_state_dict': model.module.state_dict(),  # State of the model
    'optimizer_state_dict': optimizer.state_dict(),  # State of the optimizer
    'loss': loss,  # Last recorded loss
    'scheduler_state_dict': scheduler.state_dict()  # State of the scheduler
}, 'glove_multi_head_attention_model_a_6.pth')


In [ ]:
from torch.utils.data import Dataset, DataLoader

class TestDataset(Dataset):
    def __init__(self, comments):
        self.comments = comments


    def __len__(self):
        return len(self.comments)

    def __getitem__(self, idx):
        comment = torch.tensor(self.comments[idx], dtype=torch.long)
        return comment

# Create datasets
test_dataset = TestDataset(X_test)  # Assuming train_labels is defined
# Create data loaders
batch_size = 32  # Define your batch size
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
model.eval()
test_predictions = []
with torch.no_grad():
    test_iterator = tqdm(test_loader, desc='Testing', dynamic_ncols=True)
    for comments in test_iterator:

        comments = comments.to(device)

        outputs = model(comments)


        probs = torch.sigmoid(outputs)  # Convert logits to probabilities
        predicted_labels = (probs > 0.5).float()  # Convert probabilities to predicted class (0 or 1)

        test_predictions.append(predicted_labels.cpu().numpy())

    test_predictions = np.concatenate(test_predictions, axis=0)

    predictions_df = pd.DataFrame({
        'ID': list(test_data.index),  # This should align with the index of your 'y' DataFrame
        'pred': test_predictions.squeeze()  # Make sure this is a 1D array
            })

Testing:   0%|          | 0/4181 [00:00<?, ?it/s]

In [ ]:
test_data['preds'] = predictions_df['pred']

In [ ]:
test_data

,index,string,preds
0,0,oh yes - were those evil christian mission...,0.0
1,1,he is considered a good candidate for a cyber ...,0.0
2,2,"lela , you admit no records exist to suppor...",0.0
3,3,I will take the iffy libertarian over the guy ...,1.0
4,4,should not your handle be republic of uranus ?,0.0
...,...,...,...
133777,133777,"is it better to be dead , and broke ? al...",0.0
133778,133778,"when you say speaking in code , you are add...",0.0
133779,133779,at least twice trained law enforcement officer...,0.0
133780,133780,"I have not said this before , but kizla you...",1.0


In [ ]:
predictions_df.to_csv('weighted_multi_head_attention_new.csv',index = False)

In [ ]:
# Multi-head GLove_att - training it on whole data - second attempt
for epoch in range(1):
    model.train()
    total_train_loss = 0
    metric.reset()

    for x, y,idx,weights in tqdm(train_loader):
        x, y = x.to(device), y.to(device).squeeze()
        #print(weights)
        weights=weights.to(device)
        #print(x.shape,y.shape)
        #groups = [torch.tensor(group).to(device) for group in groups]
        optimizer.zero_grad()
        outputs = model(x).squeeze()
        #print(outputs.shape)
        #break
        #print(f"x shape: {x.shape}, y shape: {y.shape}, outputs shape: {outputs.shape}")
        #print(groups)
        #break
        #loss = custom_loss(outputs, y,groups)
        loss = (F.binary_cross_entropy_with_logits(outputs,y, reduction='none') * weights ).mean()
        loss.backward()
        # Gradient clipping
        #torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        #scheduler.step()

        total_train_loss += loss.item()
        #print(torch.sigmoid(outputs.squeeze()))
        #print(y.int())
        metric.update(torch.sigmoid(outputs), y.int())

    train_accuracy = metric.compute()
    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {total_train_loss/len(train_loader):.4f}, Training Accuracy: {train_accuracy:.4f}")

    # Validation
    model.eval()

    val_loss = 0.0
    metric.reset()
    val_predictions = []
    val_indices = []

    with torch.no_grad():
        for x, y,idx,weights in tqdm(val_loader):
            x, y = x.to(device), y.to(device).squeeze()
            #groups = [torch.tensor(group).to(device) for group in groups]
            outputs = model(x).squeeze()
            val_loss += (F.binary_cross_entropy_with_logits(outputs, y, reduction='none') ).mean().item()
            metric.update(torch.sigmoid(outputs), y.int())

            val_predictions.extend(torch.sigmoid(outputs).cpu().numpy())
            val_indices.extend(idx.numpy())

    val_accuracy = metric.compute()
    val_loss /= len(val_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")



    # Format predictions for WGA calculation
    val_pred_df = pd.DataFrame({'index': val_indices, 'pred': val_predictions}).set_index('index')

    # Compute WGA
    group_accuracies, wga = compute_group_accuracies(val_pred_df, val_loader.dataset.labels)  # Ensure val_metadata is defined and aligned
    print(f"Epoch {epoch+1}/{num_epochs}, Group Accuracies:")
    for group, accuracy in group_accuracies.items():
        print(f"  {group}: {accuracy:.4f}")
    print(f"Validation WGA: {wga:.4f}")

    accuracies_list = [accuracy for group, accuracy in group_accuracies.items()]
    median_accuracy = np.median(accuracies_list)
    mean_accuracy = np.mean(accuracies_list)
    print(f'median:{median_accuracy}')
    print(f'mean:{mean_accuracy}')

# Save the model if needed
# torch.save(model.state_dict(), 'model_path.pth')


  0%|          | 0/5143 [00:00<?, ?it/s]

Epoch 1/10, Training Loss: 0.0739, Training Accuracy: 0.9789


  0%|          | 0/706 [00:00<?, ?it/s]

Epoch 1/10, Validation Loss: 0.3457, Validation Accuracy: 0.9311
Epoch 1/10, Group Accuracies:
  male_0: 0.9342
  male_1: 0.9051
  female_0: 0.9348
  female_1: 0.9066
  LGBTQ_0: 0.9323
  LGBTQ_1: 0.8943
  christian_0: 0.9300
  christian_1: 0.9413
  muslim_0: 0.9324
  muslim_1: 0.9052
  other_religions_0: 0.9318
  other_religions_1: 0.8996
  black_0: 0.9331
  black_1: 0.8801
  white_0: 0.9338
  white_1: 0.8912
Validation WGA: 0.8801
median:0.9309005866566593
mean:0.9178700760521801


## Second model(Multi-head GLove_att)

In [ ]:
# Second model Multi-head GLove_att - training it on whole data - but with a bigger glove embedding - wiki
for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0
    metric.reset()

    for x, y,idx,weights in tqdm(train_loader):
        x, y = x.to(device), y.to(device).squeeze()
        #print(weights)
        weights=weights.to(device)
        #print(x.shape,y.shape)
        #groups = [torch.tensor(group).to(device) for group in groups]
        optimizer.zero_grad()
        outputs = model(x).squeeze()
        #print(outputs.shape)
        #break
        #print(f"x shape: {x.shape}, y shape: {y.shape}, outputs shape: {outputs.shape}")
        #print(groups)
        #break
        #loss = custom_loss(outputs, y,groups)
        loss = (F.binary_cross_entropy_with_logits(outputs,y, reduction='none') * weights ).mean()
        loss.backward()
        # Gradient clipping
        #torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        #scheduler.step()

        total_train_loss += loss.item()
        #print(torch.sigmoid(outputs.squeeze()))
        #print(y.int())
        metric.update(torch.sigmoid(outputs), y.int())

    train_accuracy = metric.compute()
    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {total_train_loss/len(train_loader):.4f}, Training Accuracy: {train_accuracy:.4f}")

    # Validation
    model.eval()

    val_loss = 0.0
    metric.reset()
    val_predictions = []
    val_indices = []

    with torch.no_grad():
        for x, y,idx,weights in tqdm(val_loader):
            x, y = x.to(device), y.to(device).squeeze()
            #groups = [torch.tensor(group).to(device) for group in groups]
            outputs = model(x).squeeze()
            val_loss += (F.binary_cross_entropy_with_logits(outputs, y, reduction='none') ).mean().item()
            metric.update(torch.sigmoid(outputs), y.int())

            val_predictions.extend(torch.sigmoid(outputs).cpu().numpy())
            val_indices.extend(idx.numpy())

    val_accuracy = metric.compute()
    val_loss /= len(val_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")



    # Format predictions for WGA calculation
    val_pred_df = pd.DataFrame({'index': val_indices, 'pred': val_predictions}).set_index('index')

    # Compute WGA
    group_accuracies, wga = compute_group_accuracies(val_pred_df, val_loader.dataset.labels)  # Ensure val_metadata is defined and aligned
    print(f"Epoch {epoch+1}/{num_epochs}, Group Accuracies:")
    for group, accuracy in group_accuracies.items():
        print(f"  {group}: {accuracy:.4f}")
    print(f"Validation WGA: {wga:.4f}")

    accuracies_list = [accuracy for group, accuracy in group_accuracies.items()]
    median_accuracy = np.median(accuracies_list)
    mean_accuracy = np.mean(accuracies_list)
    print(f'median:{median_accuracy}')
    print(f'mean:{mean_accuracy}')

# Save the model if needed
# torch.save(model.state_dict(), 'model_path.pth')


  0%|          | 0/5143 [00:00<?, ?it/s]

Epoch 1/10, Training Loss: 0.3915, Training Accuracy: 0.8699


  0%|          | 0/706 [00:00<?, ?it/s]

Epoch 1/10, Validation Loss: 0.1965, Validation Accuracy: 0.9128
Epoch 1/10, Group Accuracies:
  male_0: 0.9160
  male_1: 0.8858
  female_0: 0.9153
  female_1: 0.8961
  LGBTQ_0: 0.9175
  LGBTQ_1: 0.7721
  christian_0: 0.9112
  christian_1: 0.9270
  muslim_0: 0.9184
  muslim_1: 0.7981
  other_religions_0: 0.9139
  other_religions_1: 0.8611
  black_0: 0.9218
  black_1: 0.6768
  white_0: 0.9279
  white_1: 0.6903
Validation WGA: 0.6768
median:0.9125731082021051
mean:0.8655765378192575


  0%|          | 0/5143 [00:00<?, ?it/s]

...

Epoch 9/10, Validation Loss: 0.2018, Validation Accuracy: 0.9296
Epoch 9/10, Group Accuracies:
  male_0: 0.9321
  male_1: 0.9081
  female_0: 0.9331
  female_1: 0.9065
  LGBTQ_0: 0.9320
  LGBTQ_1: 0.8586
  christian_0: 0.9280
  christian_1: 0.9440
  muslim_0: 0.9319
  muslim_1: 0.8825
  other_religions_0: 0.9303
  other_religions_1: 0.8966
  black_0: 0.9328
  black_1: 0.8438
  white_0: 0.9342
  white_1: 0.8615
Validation WGA: 0.8438
median:0.9291570785649841
mean:0.9097438696232636


  0%|          | 0/5143 [00:00<?, ?it/s]

Epoch 10/10, Training Loss: 0.1462, Training Accuracy: 0.9518


  0%|          | 0/706 [00:00<?, ?it/s]

Epoch 10/10, Validation Loss: 0.2249, Validation Accuracy: 0.9334
Epoch 10/10, Group Accuracies:
  male_0: 0.9363
  male_1: 0.9093
  female_0: 0.9366
  female_1: 0.9119
  LGBTQ_0: 0.9353
  LGBTQ_1: 0.8778
  christian_0: 0.9324
  christian_1: 0.9426
  muslim_0: 0.9354
  muslim_1: 0.8924
  other_religions_0: 0.9340
  other_religions_1: 0.9097
  black_0: 0.9362
  black_1: 0.8596
  white_0: 0.9377
  white_1: 0.8702
Validation WGA: 0.8596
median:0.9331700848532032
mean:0.9160971805250191


In [ ]:
# Save the model's state, optimizer's state, and scheduler's state to a file.
torch.save({
    'epoch': epoch,  # Current epoch number
    'model_state_dict': model.module.state_dict(),  # State of the model
    'optimizer_state_dict': optimizer.state_dict(),  # State of the optimizer
    'loss': loss,  # Last recorded loss
    'scheduler_state_dict': scheduler.state_dict()  # State of the scheduler
}, 'glove_multi_head_attention_model.pth')


In [ ]:
# Multi-head GLove_att - training it on whole data - but with a bigger glove embedding - wiki
for epoch in range(5):
    model.train()
    total_train_loss = 0
    metric.reset()

    for x, y,idx,weights in tqdm(train_loader):
        x, y = x.to(device), y.to(device).squeeze()
        #print(weights)
        weights=weights.to(device)
        #print(x.shape,y.shape)
        #groups = [torch.tensor(group).to(device) for group in groups]
        optimizer.zero_grad()
        outputs = model(x).squeeze()
        #print(outputs.shape)
        #break
        #print(f"x shape: {x.shape}, y shape: {y.shape}, outputs shape: {outputs.shape}")
        #print(groups)
        #break
        #loss = custom_loss(outputs, y,groups)
        loss = (F.binary_cross_entropy_with_logits(outputs,y, reduction='none') * weights ).mean()
        loss.backward()
        # Gradient clipping
        #torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        #scheduler.step()

        total_train_loss += loss.item()
        #print(torch.sigmoid(outputs.squeeze()))
        #print(y.int())
        metric.update(torch.sigmoid(outputs), y.int())

    train_accuracy = metric.compute()
    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {total_train_loss/len(train_loader):.4f}, Training Accuracy: {train_accuracy:.4f}")

    # Validation
    model.eval()

    val_loss = 0.0
    metric.reset()
    val_predictions = []
    val_indices = []

    with torch.no_grad():
        for x, y,idx,weights in tqdm(val_loader):
            x, y = x.to(device), y.to(device).squeeze()
            #groups = [torch.tensor(group).to(device) for group in groups]
            outputs = model(x).squeeze()
            val_loss += (F.binary_cross_entropy_with_logits(outputs, y, reduction='none') ).mean().item()
            metric.update(torch.sigmoid(outputs), y.int())

            val_predictions.extend(torch.sigmoid(outputs).cpu().numpy())
            val_indices.extend(idx.numpy())

    val_accuracy = metric.compute()
    val_loss /= len(val_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")



    # Format predictions for WGA calculation
    val_pred_df = pd.DataFrame({'index': val_indices, 'pred': val_predictions}).set_index('index')

    # Compute WGA
    group_accuracies, wga = compute_group_accuracies(val_pred_df, val_loader.dataset.labels)  # Ensure val_metadata is defined and aligned
    print(f"Epoch {epoch+1}/{num_epochs}, Group Accuracies:")
    for group, accuracy in group_accuracies.items():
        print(f"  {group}: {accuracy:.4f}")
    print(f"Validation WGA: {wga:.4f}")

    accuracies_list = [accuracy for group, accuracy in group_accuracies.items()]
    median_accuracy = np.median(accuracies_list)
    mean_accuracy = np.mean(accuracies_list)
    print(f'median:{median_accuracy}')
    print(f'mean:{mean_accuracy}')

# Save the model if needed
# torch.save(model.state_dict(), 'model_path.pth')


  0%|          | 0/5143 [00:00<?, ?it/s]

Epoch 1/10, Training Loss: 0.1321, Training Accuracy: 0.9569


  0%|          | 0/706 [00:00<?, ?it/s]

Epoch 1/10, Validation Loss: 0.2223, Validation Accuracy: 0.9268
Epoch 1/10, Group Accuracies:
  male_0: 0.9297
  male_1: 0.9022
  female_0: 0.9301
  female_1: 0.9044
  LGBTQ_0: 0.9291
  LGBTQ_1: 0.8579
  christian_0: 0.9257
  christian_1: 0.9367
  muslim_0: 0.9291
  muslim_1: 0.8787
  other_religions_0: 0.9276
  other_religions_1: 0.8884
  black_0: 0.9304
  black_1: 0.8311
  white_0: 0.9321
  white_1: 0.8479
Validation WGA: 0.8311
median:0.9266539612542917
mean:0.9050804451095609


  0%|          | 0/5143 [00:00<?, ?it/s]

...

Epoch 4/10, Validation Loss: 0.2672, Validation Accuracy: 0.9314
Epoch 4/10, Group Accuracies:
  male_0: 0.9341
  male_1: 0.9085
  female_0: 0.9349
  female_1: 0.9080
  LGBTQ_0: 0.9328
  LGBTQ_1: 0.8888
  christian_0: 0.9301
  christian_1: 0.9429
  muslim_0: 0.9328
  muslim_1: 0.9033
  other_religions_0: 0.9321
  other_religions_1: 0.9016
  black_0: 0.9337
  black_1: 0.8723
  white_0: 0.9347
  white_1: 0.8821
Validation WGA: 0.8723
median:0.9310988750058462
mean:0.9170451935384162


  0%|          | 0/5143 [00:00<?, ?it/s]

Epoch 5/10, Training Loss: 0.0934, Training Accuracy: 0.9719


  0%|          | 0/706 [00:00<?, ?it/s]

Epoch 5/10, Validation Loss: 0.2683, Validation Accuracy: 0.9298
Epoch 5/10, Group Accuracies:
  male_0: 0.9327
  male_1: 0.9051
  female_0: 0.9340
  female_1: 0.9022
  LGBTQ_0: 0.9311
  LGBTQ_1: 0.8909
  christian_0: 0.9285
  christian_1: 0.9413
  muslim_0: 0.9310
  muslim_1: 0.9047
  other_religions_0: 0.9307
  other_religions_1: 0.8895
  black_0: 0.9319
  black_1: 0.8747
  white_0: 0.9327
  white_1: 0.8873
Validation WGA: 0.8747
median:0.9296201482753792
mean:0.9155281204181197


In [ ]:
# Multi-head GLove_att - training it on whole data - but with a bigger glove embedding - wiki
for epoch in range(1):
    model.train()
    total_train_loss = 0
    metric.reset()

    for x, y,idx,weights in tqdm(train_loader):
        x, y = x.to(device), y.to(device).squeeze()
        #print(weights)
        weights=weights.to(device)
        #print(x.shape,y.shape)
        #groups = [torch.tensor(group).to(device) for group in groups]
        optimizer.zero_grad()
        outputs = model(x).squeeze()
        #print(outputs.shape)
        #break
        #print(f"x shape: {x.shape}, y shape: {y.shape}, outputs shape: {outputs.shape}")
        #print(groups)
        #break
        #loss = custom_loss(outputs, y,groups)
        loss = (F.binary_cross_entropy_with_logits(outputs,y, reduction='none') * weights ).mean()
        loss.backward()
        # Gradient clipping
        #torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        #scheduler.step()

        total_train_loss += loss.item()
        #print(torch.sigmoid(outputs.squeeze()))
        #print(y.int())
        metric.update(torch.sigmoid(outputs), y.int())

    train_accuracy = metric.compute()
    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {total_train_loss/len(train_loader):.4f}, Training Accuracy: {train_accuracy:.4f}")

    # Validation
    model.eval()

    val_loss = 0.0
    metric.reset()
    val_predictions = []
    val_indices = []

    with torch.no_grad():
        for x, y,idx,weights in tqdm(val_loader):
            x, y = x.to(device), y.to(device).squeeze()
            #groups = [torch.tensor(group).to(device) for group in groups]
            outputs = model(x).squeeze()
            val_loss += (F.binary_cross_entropy_with_logits(outputs, y, reduction='none') ).mean().item()
            metric.update(torch.sigmoid(outputs), y.int())

            val_predictions.extend(torch.sigmoid(outputs).cpu().numpy())
            val_indices.extend(idx.numpy())

    val_accuracy = metric.compute()
    val_loss /= len(val_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")



    # Format predictions for WGA calculation
    val_pred_df = pd.DataFrame({'index': val_indices, 'pred': val_predictions}).set_index('index')

    # Compute WGA
    group_accuracies, wga = compute_group_accuracies(val_pred_df, val_loader.dataset.labels)  # Ensure val_metadata is defined and aligned
    print(f"Epoch {epoch+1}/{num_epochs}, Group Accuracies:")
    for group, accuracy in group_accuracies.items():
        print(f"  {group}: {accuracy:.4f}")
    print(f"Validation WGA: {wga:.4f}")

    accuracies_list = [accuracy for group, accuracy in group_accuracies.items()]
    median_accuracy = np.median(accuracies_list)
    mean_accuracy = np.mean(accuracies_list)
    print(f'median:{median_accuracy}')
    print(f'mean:{mean_accuracy}')

# Save the model if needed
# torch.save(model.state_dict(), 'model_path.pth')


  0%|          | 0/5143 [00:00<?, ?it/s]

Epoch 1/10, Training Loss: 0.0870, Training Accuracy: 0.9738


  0%|          | 0/706 [00:00<?, ?it/s]

Epoch 1/10, Validation Loss: 0.2610, Validation Accuracy: 0.9296
Epoch 1/10, Group Accuracies:
  male_0: 0.9324
  male_1: 0.9062
  female_0: 0.9333
  female_1: 0.9046
  LGBTQ_0: 0.9310
  LGBTQ_1: 0.8874
  christian_0: 0.9288
  christian_1: 0.9369
  muslim_0: 0.9309
  muslim_1: 0.9033
  other_religions_0: 0.9301
  other_religions_1: 0.9087
  black_0: 0.9318
  black_1: 0.8711
  white_0: 0.9326
  white_1: 0.8859
Validation WGA: 0.8711
median:0.929415111215445
mean:0.9159354129203245


In [ ]:
# Multi-head GLove_att - training it on whole data - but with a bigger glove embedding - wiki
for epoch in range(1):
    model.train()
    total_train_loss = 0
    metric.reset()

    for x, y,idx,weights in tqdm(train_loader):
        x, y = x.to(device), y.to(device).squeeze()
        #print(weights)
        weights=weights.to(device)
        #print(x.shape,y.shape)
        #groups = [torch.tensor(group).to(device) for group in groups]
        optimizer.zero_grad()
        outputs = model(x).squeeze()
        #print(outputs.shape)
        #break
        #print(f"x shape: {x.shape}, y shape: {y.shape}, outputs shape: {outputs.shape}")
        #print(groups)
        #break
        #loss = custom_loss(outputs, y,groups)
        loss = (F.binary_cross_entropy_with_logits(outputs,y, reduction='none') * weights ).mean()
        loss.backward()
        # Gradient clipping
        #torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        #scheduler.step()

        total_train_loss += loss.item()
        #print(torch.sigmoid(outputs.squeeze()))
        #print(y.int())
        metric.update(torch.sigmoid(outputs), y.int())

    train_accuracy = metric.compute()
    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {total_train_loss/len(train_loader):.4f}, Training Accuracy: {train_accuracy:.4f}")

    # Validation
    model.eval()

    val_loss = 0.0
    metric.reset()
    val_predictions = []
    val_indices = []

    with torch.no_grad():
        for x, y,idx,weights in tqdm(val_loader):
            x, y = x.to(device), y.to(device).squeeze()
            #groups = [torch.tensor(group).to(device) for group in groups]
            outputs = model(x).squeeze()
            val_loss += (F.binary_cross_entropy_with_logits(outputs, y, reduction='none') ).mean().item()
            metric.update(torch.sigmoid(outputs), y.int())

            val_predictions.extend(torch.sigmoid(outputs).cpu().numpy())
            val_indices.extend(idx.numpy())

    val_accuracy = metric.compute()
    val_loss /= len(val_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")



    # Format predictions for WGA calculation
    val_pred_df = pd.DataFrame({'index': val_indices, 'pred': val_predictions}).set_index('index')

    # Compute WGA
    group_accuracies, wga = compute_group_accuracies(val_pred_df, val_loader.dataset.labels)  # Ensure val_metadata is defined and aligned
    print(f"Epoch {epoch+1}/{num_epochs}, Group Accuracies:")
    for group, accuracy in group_accuracies.items():
        print(f"  {group}: {accuracy:.4f}")
    print(f"Validation WGA: {wga:.4f}")

    accuracies_list = [accuracy for group, accuracy in group_accuracies.items()]
    median_accuracy = np.median(accuracies_list)
    mean_accuracy = np.mean(accuracies_list)
    print(f'median:{median_accuracy}')
    print(f'mean:{mean_accuracy}')

# Save the model if needed
# torch.save(model.state_dict(), 'model_path.pth')


  0%|          | 0/5143 [00:00<?, ?it/s]

Epoch 1/10, Training Loss: 0.0818, Training Accuracy: 0.9758


  0%|          | 0/706 [00:00<?, ?it/s]

Epoch 1/10, Validation Loss: 0.2907, Validation Accuracy: 0.9309
Epoch 1/10, Group Accuracies:
  male_0: 0.9336
  male_1: 0.9085
  female_0: 0.9350
  female_1: 0.9038
  LGBTQ_0: 0.9325
  LGBTQ_1: 0.8840
  christian_0: 0.9296
  christian_1: 0.9426
  muslim_0: 0.9323
  muslim_1: 0.9028
  other_religions_0: 0.9316
  other_religions_1: 0.9026
  black_0: 0.9334
  black_1: 0.8668
  white_0: 0.9337
  white_1: 0.8898
Validation WGA: 0.8668
median:0.930603848885863
mean:0.916421042585877


In [ ]:
# Save the model's state, optimizer's state, and scheduler's state to a file.
torch.save({
    'epoch': epoch,  # Current epoch number
    'model_state_dict': model.module.state_dict(),  # State of the model
    'optimizer_state_dict': optimizer.state_dict(),  # State of the optimizer
    'loss': loss,  # Last recorded loss
    'scheduler_state_dict': scheduler.state_dict()  # State of the scheduler
}, 'glove_multi_head_attention_model_1.pth')


In [ ]:
# Load checkpoint
checkpoint = torch.load('/kaggle/input/data-bal/glove_multi_head_attention_model_1.pth')

# Adjust the keys
state_dict = {f'module.{k}': v for k, v in checkpoint['model_state_dict'].items()}

# Load the adjusted state dict
model.load_state_dict(state_dict)

optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
scheduler.load_state_dict(checkpoint['scheduler_state_dict'])


In [ ]:
from torch.utils.data import Dataset, DataLoader

class TestDataset(Dataset):
    def __init__(self, comments):
        self.comments = comments


    def __len__(self):
        return len(self.comments)

    def __getitem__(self, idx):
        comment = torch.tensor(self.comments[idx], dtype=torch.long)
        return comment

# Create datasets
test_dataset = TestDataset(X_test)  # Assuming train_labels is defined
# Create data loaders
batch_size = 32  # Define your batch size
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
model.eval()
test_predictions = []
with torch.no_grad():
    test_iterator = tqdm(test_loader, desc='Testing', dynamic_ncols=True)
    for comments in test_iterator:

        comments = comments.to(device)

        outputs = model(comments)


        probs = torch.sigmoid(outputs)  # Convert logits to probabilities
        predicted_labels = (probs > 0.5).float()  # Convert probabilities to predicted class (0 or 1)

        test_predictions.append(predicted_labels.cpu().numpy())

    test_predictions = np.concatenate(test_predictions, axis=0)

    predictions_df = pd.DataFrame({
        'ID': list(test_data.index),  # This should align with the index of your 'y' DataFrame
        'pred': test_predictions.squeeze()  # Make sure this is a 1D array
            })

Testing:   0%|          | 0/4181 [00:00<?, ?it/s]

In [ ]:
test_data['preds'] = predictions_df['pred']

In [ ]:
test_data

,index,string,preds
0,0,oh yes - were those evil christian mission...,1.0
1,1,he is considered a good candidate for a cyber ...,0.0
2,2,"lela , you admit no records exist to suppor...",0.0
3,3,I will take the iffy libertarian over the guy ...,1.0
4,4,should not your handle be republic of uranus ?,0.0
...,...,...,...
133777,133777,"is it better to be dead , and broke ? al...",0.0
133778,133778,"when you say speaking in code , you are add...",0.0
133779,133779,at least twice trained law enforcement officer...,1.0
133780,133780,"I have not said this before , but kizla you...",0.0


In [ ]:
predictions_df

,ID,pred
0,0,1.0
1,1,0.0
2,2,0.0
3,3,1.0
4,4,0.0
...,...,...
133777,133777,0.0
133778,133778,0.0
133779,133779,1.0
133780,133780,0.0


In [ ]:
predictions_df.to_csv('weighted_multi_head_attention.csv',index = False)